In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Feedback or issues?
For any feedback or questions, please open an [issue](https://github.com/googleapis/python-aiplatform/issues).

# AI Platform (Unified) SDK: AutoML Image Classfication Training Example
To use this Jupyter notebook, copy the notebook to an AI Platform(Unified) Notebooks instance and open it. You can run each step, or cell, and see its results. To run a cell, use Shift+Enter. Jupyter automatically displays the return value of the last line in each cell. For more information about running notebooks in AI Platform(Unified) Notebook, see the [AI Platform(Unified) Notebook guide](https://cloud.google.com/ai-platform-unified/docs/general/notebooks).

This notebook demonstrate how to train an AutoML Image Classification model. It will require you provide a bucket where the dataset will be stored.

Note: you may incur charges for training, prediction, storage or usage of other GCP products in connection with testing this SDK.

# Install SDK
 
After the SDK installation the kernel will be automatically restarted.

In [ ]:
!pip3 uninstall -y google-cloud-aiplatform
!pip3 install google-cloud-aiplatform
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [ ]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth

    auth.authenticate_user()

### Enter your project and GCS bucket

Enter your Project Id in the cell below. Then run the cell to make sure the Cloud SDK uses the right project for all the commands in this notebook.

In [ ]:
MY_PROJECT = "YOUR PROJECT ID"
MY_STAGING_BUCKET = "gs://YOUR BUCKET"  # bucket should be in same region as ucaip

## Initialize AI Platform (Unified) SDK

Initialize the *client* for AI Platform (Unified)

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=MY_PROJECT, staging_bucket=MY_STAGING_BUCKET)

# Create Managed Image Dataset from CSV

This section will create a managed Image dataset from the Flowers dataset. For more imformation on this dataset please visit https://www.tensorflow.org/datasets/catalog/tf_flowers.

In [ ]:
IMPORT_FILE = (
    "gs://cloud-samples-data/vision/automl_classification/flowers/all_data_v2.csv"
)

ds = aiplatform.ImageDataset.create(
    display_name="flowers",
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aiplatform.schema.dataset.ioformat.image.single_label_classification,
)

ds.resource_name

# Launch a Training Job to Create a Model

Train an AutoML Image Classification model.

In [ ]:
job = aiplatform.AutoMLImageTrainingJob(
    display_name="train-iris-automl-mbsdk-1",
    prediction_type="classification",
    multi_label=False,
    model_type="CLOUD",
    base_model=None,
)

# This will take around half an hour to run
model = job.run(
    dataset=ds,
    model_display_name="iris-classification-model-mbsdk",
    training_fraction_split=0.6,
    validation_fraction_split=0.2,
    test_fraction_split=0.2,
    budget_milli_node_hours=8000,
    disable_early_stopping=False,
)

# Deploy Your Model

Deploy your model, then wait until the model FINISHES deployment before proceeding to prediction.

In [ ]:
endpoint = model.deploy()

# Predict on Endpoint
- Take one sample from the data imported to the dataset
- This sample will be encoded to base64 and passed to the endpoint for prediction

In [ ]:
test_item = !gsutil cat $IMPORT_FILE | head -n1
test_item, test_label = str(test_item[0]).split(",")

print(test_item, test_label)

In [ ]:
import base64

import tensorflow as tf
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

with tf.io.gfile.GFile(test_item, "rb") as f:
    content = f.read()

# The format of each instance should conform to the deployed model's prediction input schema.
instances_list = [{"content": base64.b64encode(content).decode("utf-8")}]
instances = [json_format.ParseDict(s, Value()) for s in instances_list]

prediction = endpoint.predict(instances=instances)

prediction

# Undeploy Model from Endpoint

In [ ]:
endpoint.undeploy_all()